In [1]:
import pandas as pd
df = pd.read_csv('data/cleaned.csv.gz', compression='gzip', low_memory=False, lineterminator='\n')
print(df.shape)
df

(157508, 13)


,id,text,create_at,geo,retweet_count,like_count,hashtags,username,following,followers,user_total_tweets,user_likes_count,cleaned_text
0,1.591219e+18,'Like a 1930s Dictator': Vitalik Buterin Lays ...,2022-11-11 23:59:59+00:00,"For now, Earth",0.0,1.0,#ftx,johnmorganFL,630.0,28492.0,240872.0,111.0,like dictator vitalik buterin lay sam bankmanf...
1,1.591219e+18,"💸 $1,000 #Giveaway 💸\n\nWe'll select 10 winner...",2022-11-11 23:59:57+00:00,Panama,8866.0,0.0,#ftx,KuKzNFT,594.0,1014.0,4452.0,8272.0,giveaway select winner randomly award account...
2,1.591219e+18,#FTX crazy Shit! #Crypto https://t.co/2GO4X3u2pt,2022-11-11 23:59:55+00:00,"Seattle, WA",0.0,0.0,#ftx,MeAndMySports,315.0,342.0,3032.0,1572.0,ftx crazy shit crypto
3,1.591219e+18,"Yo @SBF_FTX, draw me like one of your French g...",2022-11-11 23:59:53+00:00,"London, England",0.0,0.0,#ftx,thedoc7er,1513.0,296.0,4541.0,3626.0,draw like french girl ftxcrash ftx binance czb...
4,1.591219e+18,@BitcoinMagazine you you simpletons cant under...,2022-11-11 23:59:51+00:00,NaN,0.0,1.0,#ftx,jesseypaul4,123.0,44.0,317.0,81.0,simpleton understand crypto wallet like exodus...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
157503,1.591565e+18,"🔴 WAR IN 🇺🇦: FRANCE 24 reporter @cntrentF24, w...",2022-11-12 22:52:11+00:00,Hauts de France 🇫🇷 🇪🇺,20.0,0.0,#ukraine,annetteashley61,10522.0,9603.0,437088.0,291392.0,war france reporter spend lot time report ukra...
157504,1.591565e+18,People still celebrating in Kherson and I love...,2022-11-12 22:52:09+00:00,"Belgrade, Serbia, Europe",1041.0,0.0,#ukraine,svrhovac,222.0,150.0,9174.0,16675.0,people celebrate kherson love bit ukraine kherson
157505,1.591565e+18,A remarkable story #Ukraine https://t.co/eeBLc...,2022-11-12 22:52:04+00:00,Guernsey,0.0,0.0,#ukraine,TimBarker10,2668.0,1217.0,3290.0,5927.0,remarkable story ukraine
157506,1.591565e+18,#Ukraine: Relief in liberated #Kherson after 8...,2022-11-12 22:52:02+00:00,🇮🇪🇪🇺🇺🇦🎮,189.0,0.0,#ukraine,Aontaithe2021,5001.0,2376.0,392315.0,308109.0,ukraine relief liberated kherson month putin ...


### Doc2vec

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
clean_tweets = df.cleaned_text
dim = 10
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(clean_tweets)]
model = Doc2Vec(documents, vector_size=dim, window=2, min_count=1, workers=4)
doc2vecs = []
for i in range(len(clean_tweets)):
    # https://stackoverflow.com/questions/31321209/doc2vec-how-to-get-document-vectors
    doc2vecs.append(model.dv[i])
df_doc = df.copy()
df_doc[[str(i) for i in range(dim)]] = pd.DataFrame(doc2vecs, index=df.index)
df_doc

In [ ]:
df_doc.to_csv('data/doc2vec_{}.csv.gz'.format(dim), compression='gzip')

### Hidden layer of BERT

In [9]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',
                                          output_hidden_states = True, # Whether the model returns all hidden-states.
                                          )
model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [11]:
clean_tweets = df.cleaned_text
# https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/#33-creating-word-and-sentence-vectors-from-hidden-states
text = clean_tweets[0]
print(text)
marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
segments_ids = [1] * len(tokenized_text)
# Convert inputs to tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

    # Extract sentence vector
    token_vecs = hidden_states[-2][0]
    sentence_embedding = torch.mean(token_vecs, dim=0)
    print(sentence_embedding)
    print(sentence_embedding.shape)

like dictator vitalik buterin lay sam bankmanfrie ftx
tensor([-1.9123e-02,  1.7378e-01,  3.8837e-01, -1.0340e-01,  1.9767e-01,
         1.0072e-02,  6.5882e-02, -9.6792e-02, -4.0863e-01, -1.1204e-01,
         3.5201e-01, -1.2200e-01, -2.1609e-01,  1.9498e-01, -3.1077e-01,
         1.0755e+00, -5.0756e-01,  2.1503e-01, -3.5640e-01, -3.9819e-02,
         4.5718e-01, -3.0101e-02, -3.9029e-02,  1.6567e-01,  2.1629e-01,
         4.8449e-01,  1.8809e-01,  3.2697e-01, -2.6692e-01,  2.1963e-01,
         1.7477e-01,  7.2260e-02, -4.5889e-02, -3.6418e-01, -2.0349e-01,
        -9.2414e-02,  1.4672e-01,  2.8624e-01, -1.1189e-01,  8.0000e-02,
         3.1481e-03, -3.5444e-01, -2.3180e-01, -1.6364e-01, -1.2557e-01,
        -4.3402e-01, -1.8824e-01, -1.0842e-01,  1.3400e-03, -8.3181e-01,
        -7.1654e-01, -4.6065e-01, -3.4996e-01, -3.3587e-01,  1.8402e-01,
         2.3870e-01,  1.4929e-02, -2.3184e-01, -1.7462e-02,  7.8871e-02,
        -2.5795e-01, -1.0149e-01, -1.7114e-01, -2.1117e-01,  2.2362e-0

In [14]:
sentence_embedding.tolist()

[-0.019122883677482605,
 0.17377856373786926,
 0.3883650302886963,
 -0.10339684784412384,
 0.197665736079216,
 0.010072129778563976,
 0.06588219851255417,
 -0.096791572868824,
 -0.4086305797100067,
 -0.11203958094120026,
 0.3520124852657318,
 -0.12200314551591873,
 -0.21608702838420868,
 0.1949787735939026,
 -0.3107700049877167,
 1.0755492448806763,
 -0.5075608491897583,
 0.21503065526485443,
 -0.35639500617980957,
 -0.03981926664710045,
 0.4571835696697235,
 -0.030101289972662926,
 -0.03902944549918175,
 0.16567257046699524,
 0.2162877917289734,
 0.4844854474067688,
 0.18808609247207642,
 0.32696640491485596,
 -0.2669205069541931,
 0.21962547302246094,
 0.17477089166641235,
 0.07226046919822693,
 -0.04588892683386803,
 -0.3641769587993622,
 -0.2034926414489746,
 -0.09241367876529694,
 0.1467154324054718,
 0.28623998165130615,
 -0.11189183592796326,
 0.08000017702579498,
 0.0031481096521019936,
 -0.3544405996799469,
 -0.23180320858955383,
 -0.16363704204559326,
 -0.1255728155374527,
 -

In [17]:
bert = pd.DataFrame(columns=[str(i) for i in range(768)])
df.loc[len(df)] = sentence_embedding.tolist()
df

ValueError: cannot set a row with mismatched columns

### SentenceTransformer

In [12]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

#Our sentences we like to encode
sentences = clean_tweets[:1]

#Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Sentence: like dictator vitalik buterin lay sam bankmanfrie ftx
Embedding: [-1.36781447e-02  2.46694293e-02 -1.31215274e-01 -5.15292399e-02
 -8.51958394e-02  9.22903698e-03  5.32923453e-02  2.34605558e-02
 -2.70742644e-02  3.11732925e-02  4.68179025e-02 -1.99802537e-02
 -2.63234302e-02  1.77390613e-02  7.10884249e-03 -3.16139124e-02
 -3.83237787e-02  1.10480145e-01 -4.51878179e-03 -1.94068756e-02
 -3.24127823e-02 -4.68800552e-02  4.06582654e-02 -1.59789454e-02
 -6.43941686e-02 -9.32335705e-02  7.06977248e-02 -5.06424569e-02
  7.31871501e-02 -6.85119489e-03  3.22131589e-02  7.38647953e-03
 -2.21598917e-03  8.43738485e-03 -4.06829827e-02  3.74396332e-02
  3.61758210e-02  6.07829727e-02 -1.10712452e-02 -9.36119556e-02
  1.21169975e-02 -5.32565527e-02 -7.89043400e-03 -2.03219932e-02
  1.41841052e-02 -5.78347705e-02 -7.30177835e-02  2.51041185e-02
  2.17415560e-02  5.33508323e-03 -5.80064356e-02 -3.58689725e-02
  3.38780321e-03 -1.26726273e-02  5.69498129e-02 -7.74800107e-02
  2.52899751e-0

In [15]:
embedding.tolist()

[-0.013678144663572311,
 0.024669429287314415,
 -0.13121527433395386,
 -0.05152923986315727,
 -0.08519583940505981,
 0.00922903697937727,
 0.05329234525561333,
 0.02346055582165718,
 -0.027074264362454414,
 0.03117329254746437,
 0.046817902475595474,
 -0.019980253651738167,
 -0.026323430240154266,
 0.017739061266183853,
 0.007108842488378286,
 -0.03161391243338585,
 -0.0383237786591053,
 0.11048014461994171,
 -0.004518781788647175,
 -0.0194068755954504,
 -0.032412782311439514,
 -0.04688005521893501,
 0.040658265352249146,
 -0.015978945419192314,
 -0.06439416855573654,
 -0.09323357045650482,
 0.07069772481918335,
 -0.050642456859350204,
 0.07318715006113052,
 -0.006851194892078638,
 0.032213158905506134,
 0.007386479526758194,
 -0.0022159891668707132,
 0.008437384851276875,
 -0.04068298265337944,
 0.037439633160829544,
 0.03617582097649574,
 0.06078297272324562,
 -0.011071245186030865,
 -0.0936119556427002,
 0.012116997502744198,
 -0.053256552666425705,
 -0.007890434004366398,
 -0.02032